<h1> Reconocimiento de Imágenes en CIFAR10 <h1>

**4.0 Importaciones necesarias**

In [1]:
from scipy.misc import imread
import cPickle as pickle
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta
from keras.regularizers import l2
import warnings
from top_level_features import hog_features
from top_level_features import color_histogram_hsv
from top_level_features import extract_features

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce 820M (CNMeM is disabled, cuDNN not available)


**4.1 Extracción de archivos**

A continuación, se extraen los bloques correspondientes al conjunto de entrenamiento y al conjunto de pruebas por medio de la función *unpickle*, la cual retorna los datos en formato de diccionario.

In [2]:
#Función para extracción de bloques de datos
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo)
    fo.close()
    return dict

**4.2 Construcción de conjuntos de entrenamiento, prueba y validación**

Por medio de la función *load_CIFAR_One*, es posible separar, para un determinado bloque, los atributos predictores y el atributo a predecir.

In [3]:
#Separación de atributos predictores y atributo a predecir
def load_CIFAR_one(filename):
    datadict = unpickle(filename)
    X = datadict['data']
    Y = datadict['labels']
    return X, np.array(Y)

A partir de la función anterior, se procede a construir matricialmente los conjuntos de entrenamiento, prueba y validación con que se trabajará más adelante. Para ello, se implementa, la función *load_CIFAR10*. Notar que el conjunto de validación se crea seleccionando las últimas cinco mil filas del conjunto de entrenamiento.

In [4]:
#Se crean, matricialmente, conjuntos de entrenamiento, prueba y validación
def load_CIFAR10(PATH):
    #Creación de matrices de entrenamiento
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_one(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    
    #Creación de matrices de prueba
    Xts, Yts = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
    
    #Creación de matrices de validación
    #Se fusionan los conjuntos Xtr e Ytr
    training_data = Xtr
    training_data_y = Ytr.reshape((50000, 1))
    training_data = np.append(training_data, training_data_y, axis=1)

    #Luego, se extrae el conjunto de validación a partir del conjunto de entrenamiento
    Xtr = training_data[:45000, :3072]
    Ytr = training_data[:45000, 3072]
    Xv = training_data[45000:, :3072]
    Yv = training_data[45000:, 3072]
    
    return Xtr, Ytr, Xts, Yts, Xv, Yv

De esta manera, se crean las matrices Xtr e Ytr, que corresponden a las imágenes y labels de entrenamiento, Xts e Yts, similes de los anteriores para el caso de pruebas y Xv e Yv, similes de los anteriores para el caso de validación.

In [5]:
Xtr, Ytr, Xts, Yts, Xv, Yv = load_CIFAR10('.')

**4.3 Normalización de datos**

Para trabajar de forma adecuada con los datos, estos son escalados de tal forma que sigan una distribución gaussiana de media 0 y varianza 1. Para ello, se construye la función *scaler_function*.

In [6]:
def scaler_function(Xtr, Xts, Xv, scale=True):
    scaler_tr = StandardScaler(with_std=scale).fit(Xtr)
    scaler_ts = StandardScaler(with_std=scale).fit(Xts)
    scaler_v = StandardScaler(with_std=scale).fit(Xv)
    Xtr_scaled = scaler_tr.transform(Xtr)
    Xts_scaled = scaler_ts.transform(Xts)
    Xv_scaled = scaler_v.transform(Xv)
    return Xtr_scaled, Xts_scaled, Xv_scaled

Luego, bajo esta estrategia, se escalan las tres matrices originales.

In [7]:
warnings.filterwarnings('ignore')
Xtr_scaled, Xts_scaled, Xv_scaled = scaler_function(Xtr, Xts, Xv)

Sin embargo, es necesario precisar que la media de las matrices son muy cercanas a 0, pero no completamente 0, tal como se muestra a continuación:

In [8]:
Xtr_scaled.mean(axis=0)

array([ -2.83190755e-16,  -1.90901616e-16,   4.72338218e-17, ...,
         1.74231000e-17,  -2.00481607e-17,   1.44319125e-16])

In [9]:
Xts_scaled.mean(axis=0)

array([ -1.80144788e-16,   3.67983422e-17,  -1.17683641e-16, ...,
         7.66442465e-17,   6.45594689e-17,   7.08322290e-18])

In [10]:
Xv_scaled.mean(axis=0)

array([  2.23288055e-16,  -4.14335233e-17,  -7.75823850e-17, ...,
         1.57651669e-17,  -4.09894341e-17,   5.15587573e-17])

Por otro lado, las varianzas son exactamente 1: 

In [11]:
Xtr_scaled.std(axis=0)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [12]:
Xts_scaled.std(axis=0)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [13]:
Xv_scaled.std(axis=0)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

**4.4 Construcción de redes neuronales para la resolución del problema planteado**

En esta sección, se implementan diferentes arquitecturas con el fin de encontrar el modelo que logre la mayor precisión de clasificación posible en el contexto del problema planteado. Se espera poder encontrar, a lo menos, un modelo cuya precisión sobre el conjunto de validación sea mayor o igual al 50%. Las pruebas serán llevadas a cabo en un equipo con procesador Intel Core i5 4030U y 8 GB de memoria. Al no disponer de una GPU adecuada para la ejecución de los experimentos, sólo se utilizará CPU para el entrenamiento, validación y testeo de las redes.

En primer lugar, se transforman las matrices Ytr, Yts e Yv a matrices categóricas. De esta forma, cada imágen tendrá asociada un vector binario para indicar la clase a la que pertenece. Por ejemplo, si una imágen pertenece a la clase 3, su vector binario será (0, 0, 1, 0, 0, 0, 0, 0, 0, 0). La transformación se lleva a cabo a continuación:

In [14]:
Ytr = to_categorical(Ytr)
Yts = to_categorical(Yts)
Yv = to_categorical(Yv)

**4.4.1 Implementación de red *feed forward* con una capa oculta y función de activación ReLU**

La primera arquitectura con que se trabaja es una red *feedforward*. En este caso, se crea una red con una 1 capa oculta, compuesta por 50 neuronas. Además, se utiliza la función de activación de neuronas *ReLU*. Notar que se ha ajustado el parámetro *dropout* al 10%. Esto quiere decir que durante la fase de entrenamiento se desactivarán el 10% de las neuronas presentes en la capa oculta, siendo estas escogidas de forma totalmente aleatoria. Con esto, se busca evitar que el modelo se sobreajuste.

A continuación, se agrega la capa de salida, compuesta por 10 neuronas y utilizando la función de activación softmax. Luego, se compila el modelo. Una vez compilado, se procede a entrenarlo, al mismo tiempo que es optimizado. Es decir, en base a los valores que se obtienen para la función de pérdida del modelo aplicado sobre el conjunto de validación, se realizan ajustes en los pesos asociados a cada neurona. Se usa gradiente descendente estocástico con tasa de aprendizaje 0.05 como método de optimización, llevando a cabo, en total, 50 iteraciones. Se consideran batches de tamaño 32. Dado a que las clases del problemas son mutuamente excluyentes, la función de perdida escogida es *categorical crossentropy*.

In [42]:
#Se crea modelo
model = Sequential()
#Se agrega capa oculta
model.add(Dense(50, kernel_initializer='uniform', input_dim=Xtr_scaled.shape[1], activation='relu'))
#Se ajusta parámetro dropout
model.add(Dropout(0.1))
#Se agrega capa de salida
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
#Se compila el modelo
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
#Se entrena el modelo
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
#Se evalúa el modelo sobre el conjunto de validación
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 7s - loss: 1.7905 - acc: 0.3598 - val_loss: 1.7989 - val_acc: 0.3668

Se puede observar que este modelo obtiene un pobre rendimiento sobre el conjunto de validación, alcanzando sólo un 45,48% de precisión.

In [44]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precisión sobre conjunto de validación: 45.48 %


Los resultados del código anterior muestran que cada iteración de optimización demora entre 6 y 10 segundos, sumando un tiempo total de 354 segundos para el total de iteraciones.

**4.4.2 Implementación de red *feed forward* con una capa oculta y función de activación tanh**

Dados los malos resultados del modelo de la sección anterior, se implementa una nueva red, cambiando la función de activación de neuronas en la capa oculta: Ahora se utiliza *tanh*. La arquitectura base se mantiene: Una capa oculta de 50 neuronas y una capa de salida de 10 neuronas. El parámetro *dropout* se mantiene en un 10%. También se mantienen el método de optimización usado, junto con la cantidad de iteraciones y el tamaño de los batches. 

In [45]:
model = Sequential()
model.add(Dense(50, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 7s - loss: 1.8101 - acc: 0.3595 - val_loss: 1.7426 - val_acc: 0.3938

Al usar la función de activación *tanh*, la precisión sobre el conjunto de validación empeora con respecto a *ReLU* (43,26%), por lo que el rendimiento vuelve a ser pobre.

In [46]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precisión sobre conjunto de validación: 43.26 %


Sin embargo, cada iteración del proceso de optimización del modelo tuvo una duración de entre 4 y 8 segundos, siendo el tiempo total de 253 [s], por lo que se requiere de un tiempo bastante menor para entrenar el modelo definido con esta función de activación, en comparación con la arquitectura anterior.

**4.4.3 Aumentado la cantidad de neuronas en la capa oculta**

Dado a que la arquitectura implementada en la sección 4.4.1 obtuvo mejores resultados, se estudia el efecto de aumentar la cantidad de neuronas presentes en su capa oculta al doble, esto es, a 100 neuronas, manteniendo la función de activación.

In [47]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 5s - loss: 1.7816 - acc: 0.3663 - val_loss: 1.8306 - val_acc: 0.3666

Se puede ver que la precisión sobre el conjunto de validación es de un 47,2%, presentado un aumento de un 1,72% con respecto al primer modelo. Sin embargo, la precisión alcanzada sigue sin ser satisfactoria.

In [48]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precisión sobre conjunto de validación: 47.2 %


Por otro lado, cada iteración tuvo una duración de entre 5 y 6 segundos, siendo el tiempo total de entrenamiento de aproximadamente 251 [s], muy inferior al alcanzado por la arquitectura de la sección 4.4.1.

**4.4.4 Aumentando la cantidad de capas ocultas**

Se desea estudiar el efecto de agregar una segunda capa oculta a la arquitectura definida en la sección 4.4.3. Dicha capa posee las mismas características que la que creó previamente, en términos de cantidad de neuronas, función de activación y regularización.

In [18]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
# Se agrega segunda capa oculta
model.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 9s - loss: 1.7601 - acc: 0.3694 - val_loss: 1.5884 - val_acc: 0.4360
Epoch 2/50
45000/45000 [==============================] - 10s - loss: 1.5677 - acc: 0.4409 - val_loss: 1.5154 - val_acc: 0.4624
Epoch 3/50
45000/45000 [==============================] - 11s - loss: 1.4906 - acc: 0.4703 - val_loss: 1.5532 - val_acc: 0.4546
Epoch 4/50
45000/45000 [==============================] - 12s - loss: 1.4387 - acc: 0.4871 - val_loss: 1.4330 - val_acc: 0.4950
Epoch 5/50
45000/45000 [==============================] - 12s - loss: 1.4031 - acc: 0.4994 - val_loss: 1.4567 - val_acc: 0.4736
Epoch 6/50
45000/45000 [==============================] - 12s - loss: 1.3741 - acc: 0.5124 - val_loss: 1.4165 - val_acc: 0.4912
Epoch 7/50
45000/45000 [==============================] - 11s - loss: 1.3507 - acc: 0.5186 - val_loss: 1.4844 - val_acc: 0.4588
Epoch 8/50
45000/45000 [==============================] 

La precisión alcanzada por esta nueva arquitectura es de un 49,02%, siendo superior en un 1,82 % a la de la arquitectura implementada en la sección anterior. Sin embargo, tampoco se logra cumplir el objetivo planteado (precisión mayor o igual a 50% en el conjunto de validación).

In [21]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precision sobre conjunto de validacion: 49.02 %


Además, cada iteración del proceso de optimización tuvo una duración de entre 9 y 12 segundos, siendo el tiempo total de entrenamiento de 584 [s], es decir, más del doble del tiempo alcanzando en la sección 4.4.3.

**4.4.5 Usando *weight decay***

Se desea estudiar el efecto de usar *weight decay* para entrenar el modelo, buscando así disminuir aún más la posibilidad de que este se vea afectado por *overfitting*. Para ello, se reutiliza la arquitectura implementada en la sección 4.4.4, por ser la que, hasta ahora, ha conseguido los mejores resultados en términos de precisión.

In [21]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='relu',
          W_regularizer=l2(0.01))) # Se agrega regularización vía weight decay
model.add(Dropout(0.1))
model.add(Dense(100, kernel_initializer='uniform', activation='relu', W_regularizer=l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax', W_regularizer=l2(0.01))) # idem
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 9s - loss: 2.7729 - acc: 0.3509 - val_loss: 2.0085 - val_acc: 0.3872
Epoch 2/50
45000/45000 [==============================] - 9s - loss: 1.8728 - acc: 0.4132 - val_loss: 1.9105 - val_acc: 0.3676
Epoch 3/50
45000/45000 [==============================] - 9s - loss: 1.8123 - acc: 0.4245 - val_loss: 1.8729 - val_acc: 0.3954
Epoch 4/50
45000/45000 [==============================] - 9s - loss: 1.8046 - acc: 0.4306 - val_loss: 1.8384 - val_acc: 0.4258
Epoch 5/50
45000/45000 [==============================] - 9s - loss: 1.8028 - acc: 0.4333 - val_loss: 1.8391 - val_acc: 0.4144
Epoch 6/50
45000/45000 [==============================] - 9s - loss: 1.7994 - acc: 0.4354 - val_loss: 1.8364 - val_acc: 0.4232
Epoch 7/50
45000/45000 [==============================] - 9s - loss: 1.7962 - acc: 0.4351 - val_loss: 1.9303 - val_acc: 0.3920
Epoch 8/50
45000/45000 [==============================] - 8s -

La precisión alcanzada por esta nueva arquitectura es de un 41,72%, muy inferior a la obtenida por medio de la arquitectura implementada en la sección anterior. Se puede ver que la precisión sobre el conjunto de entrenamiento es también baja (45%, aproximádamente), por lo que se puede decir que si bien el modelo no se ve afectado por *overfitting*, si se ve perjudicado por el fenómeno de *underfitting*. Así, queda descartada su elección.

In [23]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precision sobre conjunto de validacion: 41.72 %


**4.4.6 Usando otras técnicas de entrenamiento**

En esta sección, se entrena el modelo usando el optimizador *Adadelta*, en vez de SGD, con el fin de observar posibles mejoras en la precisión sobre el conjunto de validación. Para esto, se reutiliza la arquitectura implementada en la sección 4.4.4. La tasa de aprendizaje, la cantidad de epochs, el número de batches y la regularización se mantienen. 

In [28]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
# Se utiliza Adadelta como método de optimización, en vez de SGD
model.compile(optimizer=Adadelta(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 9s - loss: 2.0918 - acc: 0.2586 - val_loss: 1.9246 - val_acc: 0.3230
Epoch 2/50
45000/45000 [==============================] - 10s - loss: 1.8685 - acc: 0.3403 - val_loss: 1.8112 - val_acc: 0.3586
Epoch 3/50
45000/45000 [==============================] - 11s - loss: 1.7849 - acc: 0.3678 - val_loss: 1.7527 - val_acc: 0.3816
Epoch 4/50
45000/45000 [==============================] - 11s - loss: 1.7359 - acc: 0.3851 - val_loss: 1.7097 - val_acc: 0.3928
Epoch 5/50
45000/45000 [==============================] - 10s - loss: 1.6990 - acc: 0.3974 - val_loss: 1.6771 - val_acc: 0.4056
Epoch 6/50
45000/45000 [==============================] - 10s - loss: 1.6689 - acc: 0.4086 - val_loss: 1.6511 - val_acc: 0.4162
Epoch 7/50
45000/45000 [==============================] - 11s - loss: 1.6485 - acc: 0.4169 - val_loss: 1.6310 - val_acc: 0.4234
Epoch 8/50
45000/45000 [==============================] 

La precisión alcanzada utilizado *Adadelta* como optimizador es de un 52,48%, superior en un 3,46% a la de la arquitectura implementada en la sección 4.4.4, situándose por sobre el objetivo de 50% de precisión planteado inicialmente.

In [29]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precision sobre conjunto de validacion: 52.48 %


En términos de tiempo, cada iteración tuvo una duración de entre 9 y 10 segundos, sumando un tiempo total de 525 [s], casi 1 minuto menos que el entrenamiento del modelo 4.4.4.

**4.4.7 Implementación de red convolucional**

Ante los malos resultados conseguidos en las secciones anteriores, se desea estudiar el efecto de construir el clasificador de imágenes a partir del entrenamiento de una red convolucional, arquitectura muy popular, de acuerdo a la literatura, al momento de resolver este tipo de problemas, pues las arquitecturas convolucionales asumen que los inputs que reciben son imágenes, por lo que están diseñadas especialmente para afontar problemas de este tipo. Para poder implementar de forma adecuada esta red, es necesario redimensionar previamente los conjuntos de entrenamiento, validación y prueba en base a la cantidad de imágenes pertenecientes a cada conjunto, las dimensiones de las imágenes (32 filas x 32 columnas) y la cantidad de canales asociados a cada uno (3 canales, pues se está trabajando con imágenes RGB).

In [17]:
#Dimensiones de imágenes de entrada
img_rows, img_cols = 32, 32
# Las imágenes están en formato RGB, es decir, se componen de 3 canales
img_channels = 3

#Se redimensionan conjuntos de entrenamiento, validación y pruebas
Xtr_scaled_conv = Xtr_scaled
Xv_scaled_conv = Xv_scaled
Xts_scaled_conv = Xts_scaled
Xtr_scaled_conv = Xtr_scaled_conv.reshape((Xtr_scaled.shape[0], img_rows, img_cols, img_channels))
Xv_scaled_conv = Xv_scaled_conv.reshape((Xv_scaled.shape[0], img_rows, img_cols, img_channels))
Xts_scaled_conv = Xts_scaled_conv.reshape((Xts_scaled.shape[0], img_rows, img_cols, img_channels))

Una vez redimensionados los conjuntos, se procede a implementar la red convolucional. Esta estará constituida por:

- Dos capas convolucionales 2D de 32 filtros, donde cada uno de ellos posee ancho y altura 3. Su función de activación es ReLU. Se desactivan el 25% de las neuronas.

- Una capa de reducción o *pooling*, encargada de reducir el tamaño del input recibido en un factor de 2 en cada dimensión.

- Dos capas convolucionales 2D de 64 filtros, donde cada uno de ellos posee ancho y altura 3. Su función de activación es ReLU. Se desactivan el 25% de las neuronas. 

- Una capa de reducción o *pooling*, igual a la añadida previamente.

- Una capa de tipo *flatten*, encargada de crear el vector de características a ser usado por la siguiente capa.

- Una capa densa de 512 neuronas, con función de activación ReLU. Se desactivan el 50% de las neuronas.

- La capa de salida, compuesta por 10 neuronas. Su función de activación es softmax.

In [18]:
# Se crea modelo
model = Sequential()

# Se añaden dos capas convolucionales de 32 filtros y una capa de pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=Xtr_scaled_conv.shape[1:], activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Se añaden dos capas convolucionales de 64 filtros y una capa de pooling
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Se genera vector de características
model.add(Flatten())

# Se añade capa densa
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Se añade capa de salida
model.add(Dense(10, activation='softmax'))

**4.4.7.1 Entrenamiento vía *Adadelta***

Ya definido el modelo, se procede a compilar, entrenar y evaluar el modelo en condiciones similares a las de las secciones anteriores, excepto por la cantidad de iteraciones, que se ha fijado en 15, dado a que el tiempo requerido para entrenar una red convolucional en CPU puede llegar a ser extremadamente alto. También, se usará *Adadelta* como optimizador, dada la mejora presentada por este método (modelo 4.4.6).

In [42]:
#Se compila el modelo usando el optimizador Adadelta
model.compile(loss='categorical_crossentropy', optimizer=Adadelta(lr=0.05), metrics=['accuracy'])

# Se entrena el modelo
model.fit(Xtr_scaled_conv, Ytr, batch_size=32, epochs=15, verbose=1, validation_data=(Xv_scaled_conv, Yv))

# Se evalúa el modelo
scores = model.evaluate(Xv_scaled_conv, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/15
45000/45000 [==============================] - 393s - loss: 2.1397 - acc: 0.2141 - val_loss: 1.9519 - val_acc: 0.3236
Epoch 2/15
45000/45000 [==============================] - 387s - loss: 1.9419 - acc: 0.2993 - val_loss: 1.8109 - val_acc: 0.3554
Epoch 3/15
45000/45000 [==============================] - 388s - loss: 1.8150 - acc: 0.3485 - val_loss: 1.6788 - val_acc: 0.4068
Epoch 4/15
45000/45000 [==============================] - 392s - loss: 1.7281 - acc: 0.3793 - val_loss: 1.6177 - val_acc: 0.4206
Epoch 5/15
45000/45000 [==============================] - 392s - loss: 1.6777 - acc: 0.3978 - val_loss: 1.5644 - val_acc: 0.4318
Epoch 6/15
45000/45000 [==============================] - 392s - loss: 1.6336 - acc: 0.4109 - val_loss: 1.5352 - val_acc: 0.4446
Epoch 7/15
45000/45000 [==============================] - 395s - loss: 1.5986 - acc: 0.4266 - val_loss: 1.5149 - val_acc: 0.4590
Epoch 8/15
45000/45000 [========================

In [ ]:
print 'Precision sobre conjunto de validacion (optimizador: Adadelta):', scores[1] * 100, '%'

Lamentablemente, la precisión alcanzada por la red convolucional es de sólo un 51,52%, inferior al 52,48% obtenido por el modelo 4.4.6. A lo anterior, se debe añadir el extenso tiempo de ejecución empleado por cada iteración, siendo dicho tiempo de entre 387 y 396 segundos para cada una. Así, el tiempo total requerido para el entrenamiento de la red es de 5882 segundos, esto es, cerca de 1 hora y 29 minutos.

**4.4.7.2 Entrenamiento vía SGD**

Dado a que los resultados obtenidos en la sección anterior estuvieron por debajo de lo esperado, se entrenar una réplica casi exacta del modelo de la sección 4.4.7.1, cambiando solamente SGD por *Adadelta* como método de entrenamiento.

In [19]:
#Se compila el modelo usando el optimizador SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05), metrics=['accuracy'])

# Se entrena el modelo
model.fit(Xtr_scaled_conv, Ytr, batch_size=32, epochs=15, verbose=1, validation_data=(Xv_scaled_conv, Yv))

# Se evalúa el modelo
scores = model.evaluate(Xv_scaled_conv, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/15
45000/45000 [==============================] - 825s - loss: 1.7586 - acc: 0.3606 - val_loss: 1.4528 - val_acc: 0.4706
Epoch 2/15
45000/45000 [==============================] - 829s - loss: 1.4689 - acc: 0.4762 - val_loss: 1.3342 - val_acc: 0.5206
Epoch 3/15
45000/45000 [==============================] - 723s - loss: 1.3470 - acc: 0.5193 - val_loss: 1.2562 - val_acc: 0.5534
Epoch 4/15
45000/45000 [==============================] - 381s - loss: 1.2520 - acc: 0.5566 - val_loss: 1.3016 - val_acc: 0.5500
Epoch 5/15
45000/45000 [==============================] - 379s - loss: 1.1903 - acc: 0.5792 - val_loss: 1.1244 - val_acc: 0.6060
Epoch 6/15
45000/45000 [==============================] - 379s - loss: 1.1323 - acc: 0.5992 - val_loss: 1.1418 - val_acc: 0.6020
Epoch 7/15
45000/45000 [==============================] - 382s - loss: 1.0886 - acc: 0.6168 - val_loss: 1.1774 - val_acc: 0.5948
Epoch 8/15
45000/45000 [========================

Se observa que la precisión alcanzada por la red convolucional sobre el conjunto de validación es de un 68,4%, cifra bastante superior en comparación con las precisiones anteriores, demostrando la conveniencia de la utilización de una arquitectura convolucional para atacar el problema propuesto. De hecho, lo más probable es que si se aumentara el número de iteraciones, sería posible conseguir una precisión aún mayor. Sin embargo, es necesario tener en cuenta que cada una de dichas iteraciones tiene asociado un alto tiempo de duración: 825, 829 y 723 segundos para las tres primeras, respectivamente, y entre 379 y 384 segundos para las restantes, siendo el tiempo total de entrenamiento: 6948 [s] = 1 hora y 56 minutos, aproximadamente), por lo que puede llegar un punto en que el entrenamiento del modelo sea inviable en términos de tiempo (a menos que este sea entrenado en GPU).

In [20]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precision sobre conjunto de validacion: 68.4 %


Como consecuencia del aceptable rendimiento presentado, se escoge el modelo convolucional entrenado vía SGD para ser evaluado sobre el conjunto de pruebas. 

In [28]:
scores_test = model.evaluate(Xts_scaled_conv, Yts)
print 'Precision sobre conjunto de pruebas:', scores_test[1] * 100, '%'

10000/10000 [==============================] - 36s    
Precision sobre conjunto de pruebas: 67.24 %


Así, se obtiene una precisión del 67,24% sobre el conjunto de pruebas, lo que permite asegurar que el modelo convolucional entrenado vía SGD en esta sección es el más indicado para resolver el problema de clasificación planteado, aunque sólo en comparación con las arquitecturas anteriormente utilizadas, pues el modelo está lejos de ser el más robusto.

**4.5 Cambio en el formato de representación de imágenes**

En esta sección, se estudian dos nuevos formatos de representación de los datos: El primero, corresponde a un histograma de tonos. El segundo, a descriptores HOG. En el siguiente código, se crean estas dos representaciones, junto con una tercera representación que emplea las dos anteriores de forma simultánea. Con ello, se espera poder encontrar representaciones para las imágenes tales que permitan construir modelos más robustos.

In [15]:
# Se extraen características en conjunto de entrenamiento
features1_tr = extract_features(Xtr,[hog_features])  # extrae hog features
features2_tr = extract_features(Xtr,[color_histogram_hsv])  # extrae histogramas de tonos
features3_tr = extract_features(Xtr,[hog_features, color_histogram_hsv])  # extrae todo

# Se extraen características en conjunto de prueba
features1_ts = extract_features(Xts,[hog_features])
features2_ts = extract_features(Xts,[color_histogram_hsv])
features3_ts = extract_features(Xts,[hog_features, color_histogram_hsv]) 

# Se extraen características en conjunto de validación
features1_v = extract_features(Xv,[hog_features])
features2_v = extract_features(Xv,[color_histogram_hsv])
features3_v = extract_features(Xv,[hog_features, color_histogram_hsv])

(45000, 32, 32, 3)
(45000, 32, 32, 3)
(45000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(5000, 32, 32, 3)
(5000, 32, 32, 3)
(5000, 32, 32, 3)


Si bien se esperaba poder utilizar la arquitectura implementada en la sección 4.4.7.2 para implementar los nuevos clasificadores, ello no ha sido posible, pues las redes convolucionales asumen que los inputs que reciben tienen un formato definido, esto es, imágenes de 32 x 32 píxeles y 3 canales. Sin embargo, la función *extract_features*, encargada de crear las nuevas representaciones, deja a cada imágen como un vector de 144 características, en el caso de los descriptores HOG, mientras que en el caso de los histogramas de tonos, tal vector está conformado por 10 características. En otras palabras, estas nuevas representaciones no son compatibles con el input esperado por una red convolucional, por lo que será necesario retomar alguna de las arquitecturas utilizadas en las secciones anteriores. Dado a que las arquitecturas de la secciones 4.4.4 y 4.4.6 fueron la que consiguieron las precisiones más altas después de la red convolucional, serán reutilizadas en las siguientes secciones.

** 4.5.1 Entrenamiento vía descriptores HOG **

Se define la arquitectura a utilizar.

In [16]:
model_hog = Sequential()
model_hog.add(Dense(100, input_dim=features1_tr.shape[1], kernel_initializer='uniform', activation='relu'))
model_hog.add(Dropout(0.1))
model_hog.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model_hog.add(Dropout(0.1))
model_hog.add(Dense(10, kernel_initializer='uniform', activation='softmax'))

**4.5.1.1 Entrenamiento vía *Adadelta***

Se procede a entrenar el modelo de la sección 4.4.6 a partir de la representación vía descriptores HOG.

In [17]:
model_hog.compile(optimizer=Adadelta(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_hog.fit(features1_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features1_v, Yv))
scores = model_hog.evaluate(features1_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 4s - loss: 2.2531 - acc: 0.1854 - val_loss: 2.1495 - val_acc: 0.2770
Epoch 2/50
45000/45000 [==============================] - 4s - loss: 2.0332 - acc: 0.2967 - val_loss: 1.9134 - val_acc: 0.3482
Epoch 3/50
45000/45000 [==============================] - 4s - loss: 1.8923 - acc: 0.3389 - val_loss: 1.8168 - val_acc: 0.3680
Epoch 4/50
45000/45000 [==============================] - 4s - loss: 1.8186 - acc: 0.3594 - val_loss: 1.7518 - val_acc: 0.3852
Epoch 5/50
45000/45000 [==============================] - 4s - loss: 1.7671 - acc: 0.3727 - val_loss: 1.7079 - val_acc: 0.3956
Epoch 6/50
45000/45000 [==============================] - 4s - loss: 1.7296 - acc: 0.3847 - val_loss: 1.6733 - val_acc: 0.4130
Epoch 7/50
45000/45000 [==============================] - 4s - loss: 1.6996 - acc: 0.3980 - val_loss: 1.6474 - val_acc: 0.4196
Epoch 8/50
45000/45000 [==============================] - 4s -

Luego, se obtiene una precisión de validación de 50,98%, menor a la de la red convolucional y a la de la arquitectura que utiliza la representación original de las imágenes (modelo 4.4.6).

In [18]:
print 'Precision sobre conjunto de validacion (Caso: Descriptores HOG; Optimizador: Adadelta):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Descriptores HOG; Optimizador: Adadelta): 50.98 %


Además, cada iteración tuvo un duración de 4 segundos, siendo el tiempo total de entrenamiento de 200 segundos, muy inferior a los 584 segundos empleados por la arquitectura 4.4.6.

**4.5.1.2 Entrenamiento vía SGD**

Se procede a entrenar el modelo de la sección 4.4.4 a partir de la misma representación de los datos.

In [19]:
model_hog.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_hog.fit(features1_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features1_v, Yv))
scores = model_hog.evaluate(features1_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 3s - loss: 1.4784 - acc: 0.4846 - val_loss: 1.5466 - val_acc: 0.4532
Epoch 2/50
45000/45000 [==============================] - 3s - loss: 1.3942 - acc: 0.5141 - val_loss: 1.3950 - val_acc: 0.5142
Epoch 3/50
45000/45000 [==============================] - 3s - loss: 1.3450 - acc: 0.5304 - val_loss: 1.3339 - val_acc: 0.5260
Epoch 4/50
45000/45000 [==============================] - 3s - loss: 1.3025 - acc: 0.5458 - val_loss: 1.3592 - val_acc: 0.5114
Epoch 5/50
45000/45000 [==============================] - 3s - loss: 1.2771 - acc: 0.5519 - val_loss: 1.3289 - val_acc: 0.5394
Epoch 6/50
45000/45000 [==============================] - 3s - loss: 1.2528 - acc: 0.5598 - val_loss: 1.4007 - val_acc: 0.5228
Epoch 7/50
45000/45000 [==============================] - 3s - loss: 1.2318 - acc: 0.5664 - val_loss: 1.3347 - val_acc: 0.5342
Epoch 8/50
45000/45000 [==============================] - 3s -

Se puede ver como la precisión sobre el conjunto de validación es de un 54,3%, es decir, nuevamente es menor a la de la red convolucional, pero superior a la de la red entrenada sobre la representación original (modelo 4.4.4).

In [20]:
print 'Precision sobre conjunto de validacion (Caso: Descriptores HOG; Optimizador: SGD):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Descriptores HOG; Optimizador: SGD): 54.3 %


Por otro lado, el tiempo de cómputo se reduce, siendo la duración de cada iteración de entre 4 y 5 segundos, sumando un tiempo total de 202 [s], muy inferior a los 525 [s] alcanzados sobre la representación original. 

**4.5.2 Entrenamiento vía histogramas de tonos** 

Se define la arquitectura a utilizar.

In [21]:
model_col_hist = Sequential()
model_col_hist.add(Dense(100, input_dim=features2_tr.shape[1], kernel_initializer='uniform', activation='relu'))
model_col_hist.add(Dropout(0.1))
model_col_hist.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model_col_hist.add(Dropout(0.1))
model_col_hist.add(Dense(10, kernel_initializer='uniform', activation='softmax'))

**4.5.2.1 Entrenamiento vía *Adadelta***

Usando la representación mediante histogramas de tonos, se vuelve a entrenar el modelo de la sección 4.4.6.

In [30]:
model_col_hist.compile(optimizer=Adadelta(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_col_hist.fit(features2_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features2_v, Yv))
scores = model_col_hist.evaluate(features2_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 4s - loss: 1.9559 - acc: 0.2776 - val_loss: 1.9399 - val_acc: 0.2816
Epoch 2/50
45000/45000 [==============================] - 3s - loss: 1.9514 - acc: 0.2791 - val_loss: 1.9395 - val_acc: 0.2810
Epoch 3/50
45000/45000 [==============================] - 3s - loss: 1.9525 - acc: 0.2766 - val_loss: 1.9391 - val_acc: 0.2812
Epoch 4/50
45000/45000 [==============================] - 4s - loss: 1.9516 - acc: 0.2808 - val_loss: 1.9389 - val_acc: 0.2794
Epoch 5/50
45000/45000 [==============================] - 4s - loss: 1.9503 - acc: 0.2818 - val_loss: 1.9388 - val_acc: 0.2814
Epoch 6/50
45000/45000 [==============================] - 4s - loss: 1.9517 - acc: 0.2783 - val_loss: 1.9387 - val_acc: 0.2802
Epoch 7/50
45000/45000 [==============================] - 4s - loss: 1.9504 - acc: 0.2796 - val_loss: 1.9387 - val_acc: 0.2804
Epoch 8/50
45000/45000 [==============================] - 4s -

Analizando los resultados, se puede ver como la precisión sobre el conjunto de validación es muy pobre (28,26%), siendo totalmente inferior a cualquiera de las arquitecturas antes implementadas. Observando que la precisión sobre el conjunto de entrenamiento también es baja (28,17%), se está ante un modelo que sufre de *underfitting*, esto es, la arquitectura no es capaz de comprender la forma en que se relacionan las variables, lo que resulta razonable considerando que la representación utilizada cuenta con sólo 10 características, probablemente una cantidad insuficiente para construir un buen clasificador.

In [33]:
print 'Precision sobre conjunto de validacion (Caso: Histogramas de colores; Optimizador: Adadelta):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Histogramas de colores; Optimizador: Adadelta): 28.26 %


**4.5.2.2 Entrenamiento vía SGD**

Usando la misma representación de la sección 4.5.2.1, se entrena el modelo 4.4.4, buscando comprobar la hipótesis de *underfitting* señalada en la sección anterior.

In [34]:
model_col_hist.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_col_hist.fit(features2_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features2_v, Yv))
scores = model_col_hist.evaluate(features2_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 10s - loss: 1.9580 - acc: 0.2784 - val_loss: 1.9452 - val_acc: 0.2768
Epoch 2/50
45000/45000 [==============================] - 10s - loss: 1.9590 - acc: 0.2758 - val_loss: 1.9433 - val_acc: 0.2828
Epoch 3/50
45000/45000 [==============================] - 9s - loss: 1.9587 - acc: 0.2770 - val_loss: 1.9487 - val_acc: 0.2706
Epoch 4/50
45000/45000 [==============================] - 10s - loss: 1.9579 - acc: 0.2759 - val_loss: 1.9400 - val_acc: 0.2772
Epoch 5/50
45000/45000 [==============================] - 10s - loss: 1.9580 - acc: 0.2758 - val_loss: 1.9410 - val_acc: 0.2794
Epoch 6/50
45000/45000 [==============================] - 11s - loss: 1.9574 - acc: 0.2762 - val_loss: 1.9524 - val_acc: 0.2766
Epoch 7/50
45000/45000 [==============================] - 10s - loss: 1.9577 - acc: 0.2763 - val_loss: 1.9463 - val_acc: 0.2756
Epoch 8/50
45000/45000 [==============================] 

Observando los resultados obtenidos, se ve que la precisión sobre el conjunto de validación es de un 28,56%, mientras que sobre el conjunto de entrenamiento, un 28,1%, por lo que este modelo también sufre de *underfitting*. De esta manera, se presenta un comportamiento análogo al entrenamiento vía *Adadelta*, lo que permite corroborar que la representación vía histograma de tonos ofrece muy pocas características como para construir modelos robustos. 

In [36]:
print 'Precision sobre conjunto de validacion (Caso: Histogramas de colores; Optimizador: SGD):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Histogramas de colores; Optimizador: SGD): 28.56 %


**4.5.3 Entrenamiento vía combinación de representaciones**

En esta sección, se utiliza la representación combinada de descriptores HOG e histogramas de color. La arquitectura a utilizar queda definida a continuación:

In [38]:
model_comb = Sequential()
model_comb.add(Dense(100, input_dim=features3_tr.shape[1], kernel_initializer='uniform', activation='relu'))
model_comb.add(Dropout(0.1))
model_comb.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model_comb.add(Dropout(0.1))
model_comb.add(Dense(10, kernel_initializer='uniform', activation='softmax'))

**4.5.3.1 Entrenamiento vía *Adadelta***

Por medio de la representación combinada, se entrena el modelo 4.4.6.

In [39]:
model_comb.compile(optimizer=Adadelta(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_comb.fit(features3_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features3_v, Yv))
scores = model_comb.evaluate(features3_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 12s - loss: 2.2481 - acc: 0.1455 - val_loss: 2.1357 - val_acc: 0.2234
Epoch 2/50
45000/45000 [==============================] - 11s - loss: 2.0228 - acc: 0.2761 - val_loss: 1.9107 - val_acc: 0.3236
Epoch 3/50
45000/45000 [==============================] - 12s - loss: 1.8941 - acc: 0.3280 - val_loss: 1.8188 - val_acc: 0.3628
Epoch 4/50
45000/45000 [==============================] - 12s - loss: 1.8218 - acc: 0.3549 - val_loss: 1.7548 - val_acc: 0.3824
Epoch 5/50
45000/45000 [==============================] - 12s - loss: 1.7673 - acc: 0.3713 - val_loss: 1.7048 - val_acc: 0.4004
Epoch 6/50
45000/45000 [==============================] - 12s - loss: 1.7279 - acc: 0.3884 - val_loss: 1.6653 - val_acc: 0.4122
Epoch 7/50
45000/45000 [==============================] - 12s - loss: 1.6914 - acc: 0.3995 - val_loss: 1.6330 - val_acc: 0.4246
Epoch 8/50
45000/45000 [==============================]

Los resultados muestran que se alcanza una precisión de 53,42% sobre el conjunto de validación, superior en un 0,94% a la precisión de la arquitectura de la sección 4.4.6, pero inferior en un 14,98% a la precisión de la red convolucional. 

In [41]:
print 'Precision sobre conjunto de validacion (Caso: Representaciones combinadas, optimizador: Adadelta):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Representaciones combinadas, optimizador: Adadelta): 53.42 %


En términos de tiempo, cada iteración tiene una duración de entre 11 y 15 segundos, alcanzando un tiempo total de 599 segundos, superior al transcurrido en el entrenamiento del modelo 4.4.6 en 74 segundos.

**4.5.3.2 Entrenamiento vía SGD**

Por medio de la representación combinada, se entrena el modelo 4.4.4.

In [42]:
model_comb.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_comb.fit(features3_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features3_v, Yv))
scores = model_comb.evaluate(features3_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 10s - loss: 1.4244 - acc: 0.4980 - val_loss: 1.4727 - val_acc: 0.4788
Epoch 2/50
45000/45000 [==============================] - 11s - loss: 1.3402 - acc: 0.5287 - val_loss: 1.3303 - val_acc: 0.5372
Epoch 3/50
45000/45000 [==============================] - 10s - loss: 1.2905 - acc: 0.5460 - val_loss: 1.4115 - val_acc: 0.5146
Epoch 4/50
45000/45000 [==============================] - 11s - loss: 1.2530 - acc: 0.5596 - val_loss: 1.3744 - val_acc: 0.5224
Epoch 5/50
45000/45000 [==============================] - 9s - loss: 1.2209 - acc: 0.5682 - val_loss: 1.2768 - val_acc: 0.5532
Epoch 6/50
45000/45000 [==============================] - 10s - loss: 1.2019 - acc: 0.5755 - val_loss: 1.2698 - val_acc: 0.5642
Epoch 7/50
45000/45000 [==============================] - 10s - loss: 1.1828 - acc: 0.5817 - val_loss: 1.2767 - val_acc: 0.5604
Epoch 8/50
45000/45000 [==============================] 

Los resultados muestran que se alcanza una precisión del 56% sobre el conjunto de validación, superior en un 6,98% a la precisión de la arquitectura de la sección 4.4.4, pero inferior en un 11,4% a la precisión de la red convolucional. 

In [44]:
print 'Precision sobre conjunto de validacion (Caso: Representaciones combinadas, optimizador: SGD):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Representaciones combinadas, optimizador: SGD): 56.0 %


En términos de tiempo, cada iteración tiene una duración de entre 11 y 15 segundos, alcanzando un tiempo total de 525 segundos, inferior al transcurrido en el entrenamiento del modelo 4.4.4 en 59 segundos.

De lo anterior, se desprende que la representación combinada entrenada vía SGD es la arquitectura que alcanza la mejor precisión sobre el conjunto de validación en esta sección (de ahora en adelante, modelo 4.5.3.2). Por ende, se elige esta arquitectura para ser evaluado sobre el conjunto de pruebas.

In [47]:
scores_test = model_comb.evaluate(features3_ts, Yts)
print ''
print 'Precision sobre conjunto de pruebas (Caso: Representaciones combinadas, optimizador: SGD):', scores_test[1] * 100, '%'

 9664/10000 [===========================>..] - ETA: 0s
Precision sobre conjunto de pruebas (Caso: Representaciones combinadas, optimizador: SGD): 55.5 %


Se observa que la precisión alcanzada sobre el conjunto de pruebas es de un 55,5%, muy inferior a la alcanzada por la red convolucional (67,24%).

**4.6 Conclusiones**

Los modelos implementados en las secciones 4.4.1, 4.4.2, 4.4.3 alcanzan un pobre rendimiento sobre el conjunto de validación. Además, el rendimiento sobre el conjunto de entrenamiento fue superior en un 20% aproximadamente en cada uno de los casos. De esta situación se desprende que tales modelos están sobreajustados y por tanto son incapaces de generalizar el problema a partir de los datos de entrenamiento. El modelo 4.4.5, por otro lado, se vió afectado por *underfitting*. El modelo 4.4.6, alcanzó el mejor rendimiento en comparación con las arquitecturas implementadas hasta ese momento gracias al cambio en el optimizador utilizado para entrenar el modelo (*Adadelta*, en vez de *SGD*). Sin embargo, las arquitecturas posteriores obtuvieron mejores precisiones al ser entrenadas con SGD. 

Por otro lado, la red convolucional construída en la sección 4.4.7.2 alcanzó una precisión más que aceptable, tanto sobre el conjunto de validación como de pruebas, siendo dichas precisiones muy cercanas a la obtenida sobre el conjunto de entrenamiento, lo que quiere decir que se redujo en parte el fenómeno de *overfitting*. Ahora bien, los tiempos de ejecución aumentaron notablemente, pero se reitera que esto pueda evitarse si se cuenta con una GPU adecuada para realizar el entrenamiento.

El modelo 4.5.3.2 alcanzó mejor rendimiento con respecto a las arquitecturas utilizadas en las secciones 4.4.1 - 4.4.6 en términos de precisión, además de presentar un sobreajuste menor (diferencias de aproximadamente 10% entre precisiones de validación y entrenamiento, en cada caso). Sin embargo, la precisión obtenida por la arquitectura 4.5.3.2 sobre el conjunto de pruebas es muy inferior a la de la red convolucional.

De esta manera, la red convolucional se identifica como la arquitectura más adecuada para resolver el problema, no obstante, se debe contar con los recursos computacionales adecuados para que su implementación pueda ser llevada a cabo en forma eficiente. En caso contrario, se debe utilizar una red *feedforward* y utilizar alguna representación que reduzca la cantidad de características asociadas a cada imágen, a costa de un bajo rendimiento sobre ejemplos nuevos.